# Пример разбора словаря

https://scipy-cookbook.readthedocs.io/items/Reading_Custom_Text_Files_with_Pyparsing.html

In [1]:
import pyparsing as pp
import numpy as np
filename = 'parsing_example/data3.txt'

## Разбор строк вида "переменная (размерность) = значение"

In [2]:
keyName = pp.Word(pp.alphanums + '_')
unitDef = pp.Suppress('(') + pp.Word(pp.alphanums + '^*/-._') + pp.Suppress(')')
paramValueDef = pp.SkipTo('#' | pp.lineEnd)

paramDef = keyName('name') + pp.Optional(unitDef)('unit') + pp.Suppress('='+pp.empty) + paramValueDef('value')

In [3]:
with open(filename) as fin:
    for param in paramDef.searchString(fin.read()):
        print(param.dump())
        print('...')

['Debug', 'False']
- name: 'Debug'
- value: 'False'
...
['Shape', 'mm^-1', '2.3']
- name: 'Shape'
- unit: ['mm^-1']
- value: '2.3'
...
['Length', 'mm', '25361.15']
- name: 'Length'
- unit: ['mm']
- value: '25361.15'
...
['1', 'C:\\\\This\\is\\a\\long\\path\\with some space in it\\data.txt']
- name: '1'
- value: 'C:\\\\This\\is\\a\\long\\path\\with some space in it\\data.txt'
...
['description', 'raw values can have multiple lines, but additional lines must start']
- name: 'description'
- value: 'raw values can have multiple lines, but additional lines must start'
...
['Parent', 'None']
- name: 'Parent'
- value: 'None'
...


Проблемы:
* `Path 1` разобрано как `1`
* description должно иметь значение из 2 строк, учтена только одна
* числа, True/False, None - все являются строками

## Регулярные выражения

Небольшое выражение иногда проще разобрать с помощью регулярного выражения.

Разберем, как это делается на примере разбора чисел: целых, с плавающей точкой и быть может экспонентой.

Для этого воспользуемся понятием **именованной группы** [named capturing group](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Regular_expressions/Named_capturing_group)
которая обозначается как

В [python](https://docs.python.org/3/library/re.html#regular-expression-syntax
) именованная группа обозначается так:

Выражение


A non-capturing version of regular parentheses.
Matches whatever regular expression is inside the parentheses,
but the substring matched by the group cannot be retrieved after performing a match or referenced later in the patter...)

In [3]:
from re        import VERBOSE
number = pp.Regex(r"""
        [+-]?                           # optional sign
         (
            (?:\d+(?P<float1>\.\d*)?)   # match 2 or 2.02
          |                             # or
            (?P<float2>\.\d+)           # match .02
         )
         (?P<float3>[Ee][+-]?\d+)?      # optional exponent
        """, flags=VERBOSE
        )
text = '1   2   3.0  0.3 .3 4. 2e2  -.2e+2 +2.2256E-2 hello 5 . 6'
# hello и отдельная точка не являются числами
print(number.searchString(text))

[['1'], ['2'], ['3.0'], ['0.3'], ['.3'], ['4.'], ['2e2'], ['-.2e+2'], ['+2.2256E-2'], ['5'], ['6']]


## Преобразуем числа из строк в int и float

* it should accepts a `parseResults` object as input value (some functions can accepts 3 parameters, see `setParseAction` documentation). 
A `parseResults` object can be used as a list, as a dict or directly with a named attribute if you have named your results. 
Here we had set three named group float1, float2 and float3 and we can use them to decide whether to use int() or float().
* it should return either a `parseResults` object or a list of results which will be automatically converted to a `parseResults` object.


In [4]:
def convertNumber(t: pp.ParseResults):
    """Convert a string matching a number to a python number"""
    # print(f'{t=}')
    if t.float1 or t.float2 or t.float3 : return [float(t[0])]
    else                                : return [int(t[0])  ]

number.setParseAction(convertNumber)

text = '1   2   3.0  0.3 .3 4. 2e2  -.2e+2 +2.2256E-2 hello 5 . 6'
print(number.searchString(text))
# теперь числа типа int или float

[[1], [2], [3.0], [0.3], [0.3], [4.0], [200.0], [-20.0], [0.022256], [5], [6]]


В реальной жизни рекомендуем воспользоваться классом [pyparsing.pyparsing_common.number](https://pythonhosted.org/pyparsing/pyparsing.pyparsing_common-class.html)

## Заменим константы на значения нужных типов: True, False, NaN, None

[MatchFirst](https://pythonhosted.org/pyparsing/pyparsing.MatchFirst-class.html) - Requires that at least one ParseExpression is found. If two expressions match, the first one listed is the one that will match. May be constructed using the `|` operator.
Наиболее специфичное надо ставить первым в цепочке перечислений, иначе будет как в примере ниже:


In [7]:
from pyparsing import Word, Combine, nums

# watch the order of expressions to match
number_ex = Word(nums) | Combine(Word(nums) + '.' + Word(nums))
print(number_ex.searchString("123 3.1416 789")) #  Fail! -> [['123'], ['3'], ['1416'], ['789']]

# put more selective expression first
number_ex = Combine(Word(nums) + '.' + Word(nums)) | Word(nums)
print(number_ex.searchString("123 3.1416 789")) #  Better -> [['123'], ['3.1416'], ['789']]


[['123'], ['3'], ['1416'], ['789']]
[['123'], ['3.1416'], ['789']]


Заметим, что сначала описываем закавыченные строки с `"""` и `'''`, потом только с одиночными кавычками.

In [5]:
pyValue_list = [ number                                                        ,
                 pp.Keyword('True').setParseAction(pp.replaceWith(True))             ,
                 pp.Keyword('False').setParseAction(pp.replaceWith(False))           ,
                 pp.Keyword('NAN', caseless=True).setParseAction(pp.replaceWith(np.NAN)),
                 pp.Keyword('None').setParseAction(pp.replaceWith(None))             ,
                 pp.QuotedString('"""', multiline=True)                           ,
                 pp.QuotedString("'''", multiline=True)                           ,
                 pp.QuotedString('"')                                             ,
                 pp.QuotedString("'")                                             ,
               ]
pyValue     = pp.MatchFirst(pyValue_list)    # тот же результат, что у предыдущей строки


In [6]:
pyValue_list_n = [ number                                                        ,
                 pp.Keyword('True').setParseAction(pp.replaceWith(True))             ,
                 pp.Keyword('False').setParseAction(pp.replaceWith(False))           ,
                 pp.Keyword('NAN', caseless=True).setParseAction(pp.replaceWith(np.NAN)),
                 pp.Keyword('None').setParseAction(pp.replaceWith(None))             ,
                 pp.QuotedString('"""', multiline=True)                           ,
                 pp.QuotedString("'''", multiline=True)                           ,
                 pp.QuotedString('"')                                             ,
                 pp.QuotedString("'")                                             ,
               ]
# после того, как мы для всех элементов удалили \n из незначащих, мы везде далее стали обращать на перводы строк внимание.
# Н
pyValue_n   = pp.MatchFirst( e.setWhitespaceChars(' \t\r') for e in pyValue_list_n)

In [7]:
test2 = '''
    1   2   3.0  0.3 .3  2e2  -.2e+2 +2.2256E-2
    True False None
    "word" "two words"
    """'more words', he said"""
    """Good bye, my love,
    Good byyyyeeee!!!"""
'''
print(pyValue.searchString(test2))
print(pyValue_n.searchString(test2))  # идентичный результат

[[1], [2], [3.0], [0.3], [0.3], [200.0], [-20.0], [0.022256], [True], [False], [None], ['word'], ['two words'], ["'more words', he said"], ['Good bye, my love,\n    Good byyyyeeee!!!']]
[[1], [2], [3.0], [0.3], [0.3], [200.0], [-20.0], [0.022256], [True], [False], [None], ['word'], ['two words'], ["'more words', he said"], ['Good bye, my love,\n    Good byyyyeeee!!!']]


По умолчанию пробельные символы `' \n\t\r'` не имеют значения. Если нужно определять конец строки, то нужно удалить `\n` из списка "незначащих" символов, используя `setWhitespaceChars` или `setDefaultWhitespaceChars`

Если мы собираемся обрабатывать таблицы построчно, нам надо это настроить на *самом низком* уровне:

In [15]:
# все разобранные части группируются в один список, так как мы считаем входной текст единым, 
# с точностью до многострочных комментариев
print(pp.OneOrMore(pyValue).searchString(test2))

[[1, 2, 3.0, 0.3, 0.3, 200.0, -20.0, 0.022256, True, False, None, 'word', 'two words', "'more words', he said", 'Good bye, my love,\n    Good byyyyeeee!!!']]


In [16]:
# все разобранные части группируются в списки ПО СТРОКАМ, так как мы считаем входной текст набором строк, 
# с точностью до многострочных комментариев
print(pp.OneOrMore(pyValue_n).searchString(test2))

[[1, 2, 3.0, 0.3, 0.3, 200.0, -20.0, 0.022256], [True, False, None], ['word', 'two words'], ["'more words', he said"], ['Good bye, my love,\n    Good byyyyeeee!!!']]


In [12]:
# список результатов сгруппирован по СТРОКАМ исходного текста
for a in pp.OneOrMore(pyValue_n).searchString(test2):
    print(a)

[1, 2, 3.0, 0.3, 0.3, 200.0, -20.0, 0.022256]
[True, False, None]
['word', 'two words']
["'more words', he said"]
['Good bye, my love,\n    Good byyyyeeee!!!']


## Имена переменных: заменяем пробелы на _

Имена переменных ограничены в конце `=`, поэтому мы можем допустить в них пробелы. Чтобы обращаться дальше по имени переменной, пробелы в них допускать нельзя. Поэтому заменим пробелы на `_`.

`downcaseTokens` - ко всему токену применяется функция `tolower`. Еще раз подчеркиваем полезность класса [pyparsing_common](https://pythonhosted.org/pyparsing/pyparsing.pyparsing_common-class.html)

In [8]:
downcaseTokens = pp.pyparsing_common.downcaseTokens
def variableParser(escapedChars, baseChars=pp.alphanums):
    """ Return pattern matching any characters in baseChars separated by
    characters defined in escapedChars. Thoses characters are replaced with '_'

    The '_' character is therefore automatically in escapedChars.
    """
    escapeDef = pp.Word(escapedChars + '_').setParseAction(pp.replaceWith('_'))
    whitespaceChars = ''.join( x for x in ' \t\r' if not x in escapedChars )
    escapeDef = escapeDef.setWhitespaceChars(whitespaceChars)
    # почему не 
    # return Combine(pp.Word(baseChars) + pp.ZeroOrMore(escapeDef + pp.Word(baseChars)))
    return pp.Combine(pp.Word(baseChars) + pp.Optional(pp.OneOrMore(escapeDef + pp.Word(baseChars))))

keyName             = variableParser(' _-./').setParseAction(downcaseTokens)
keyNameWithoutSpace = variableParser('_-./').setParseAction(downcaseTokens)

In [18]:
# вспомним, когда мы их использовали в самом начале и прикрутим туда новое определение keyName
unitDef = pp.Suppress('(') + pp.Word(pp.alphanums + '^*/-._') + pp.Suppress(')')
paramValueDef = pp.SkipTo('#' | pp.lineEnd)

paramDef = keyName('name') + pp.Optional(unitDef)('unit') + pp.Suppress('='+pp.empty) + paramValueDef('value')

with open(filename) as fin:
    for param in paramDef.searchString(fin.read()):
        print(param.dump())
        print('...')
# 'Path 1' теперь именуется не как '1', а как path_1

['debug', 'False']
- name: 'debug'
- value: 'False'
...
['shape', 'mm^-1', '2.3']
- name: 'shape'
- unit: ['mm^-1']
- value: '2.3'
...
['length', 'mm', '25361.15']
- name: 'length'
- unit: ['mm']
- value: '25361.15'
...
['path_1', 'C:\\\\This\\is\\a\\long\\path\\with some space in it\\data.txt']
- name: 'path_1'
- value: 'C:\\\\This\\is\\a\\long\\path\\with some space in it\\data.txt'
...
['description', 'raw values can have multiple lines, but additional lines must start']
- name: 'description'
- value: 'raw values can have multiple lines, but additional lines must start'
...
['parent', 'None']
- name: 'parent'
- value: 'None'
...


В `paramValueDef = pp.SkipTo('#' | pp.lineEnd)` надо с одной стороны разобрать числа и True/False/None и тп, как мы научились делать в `pyValue`. С другой стороны не потерять значения в виде строк до конца строки или комментария.

Определим `rawValue` - все, что не попадает в `pyValue`, по следующим правилам:

* все, что находится после символа `#`, считаем комментарием и пропускаем.
* необработанное значение может быть на нескольких строках, но дополнительные строки должны начинаться с пробела, а не с `[`, ибо с `[` начинается заголовок секции.


In [9]:
# rawValue can be multiline but theses lines should start with a Whitespace
rawLine  = pp.CharsNotIn('#\n') + (pp.lineEnd | pp.Suppress('#'+pp.restOfLine))
rawValue = pp.Combine( rawLine + pp.ZeroOrMore(pp.White(' \t').suppress()+ pp.NotAny('[') + rawLine))
rawValue.setParseAction(lambda t: [x.strip() for x in t])

Combine:({{!W:(#
) {line_end | Suppress:({'#' rest of line})}} [{Suppress:(<SP><TAB>) ~{'['}} {!W:(#
) {line_end | Suppress:({'#' rest of line})}}]...})

Единицы измерения. Добавим обработку особых случаев: `(-)`, `(/)`, `()` и `( )`. Будем считать, что единицы измерения - строка.

In [10]:
# было
unitDef = pp.Suppress('(') + pp.Word(pp.alphanums + '^*/-._') + pp.Suppress(')')
# стало
unitDef  = pp.Suppress('(') + (pp.Suppress(pp.oneOf('- /')) | pp.Optional(pp.Word(pp.alphanums + '^*/-._'))) + pp.Suppress(')')

Полное определение переменной, единиц изменения и значения:

In [11]:
unitDef  = pp.Suppress('(') + (pp.Suppress(pp.oneOf('- /')) | pp.Optional(pp.Word(pp.alphanums + '^*/-._'))) + pp.Suppress(')')
valueDef = pyValue | rawValue
paramDef = keyName('name') + pp.Optional(unitDef)('unit') + pp.Suppress("="+pp.empty) + valueDef('value')

## Структурируем полученные результаты разбора

[Dict](https://pythonhosted.org/pyparsing/pyparsing.Dict-class.html) Конвертер для возврата повторяющегося выражения в виде списка, а также словаря. На каждый элемент можно также ссылаться, используя первый токен в выражении в качестве его ключа. Полезно для скрапинга табличных отчетов, когда первый столбец может использоваться в качестве ключа элемента.

In [12]:
def formatBloc(t):
    """ Format the result to have a list of (key, values) easily usable with Dict

    Add two fields :
        names_ : the list of column names found
        units_ : a dict in the form {key : unit}
    """
    rows = []

    # store units and names
    units = {}
    names = []

    for row in t :
        rows.append(pp.ParseResults([ row.name, row.value ]))
        names.append(row.name)
        if row.unit : units[row.name] = row.unit[0]

    rows.append( pp.ParseResults([ 'names_', names ]))
    rows.append( pp.ParseResults([ 'unit_',  units]))

    return rows

paramParser = pp.Dict( pp.OneOrMore( pp.Group(paramDef)).setParseAction(formatBloc))

In [13]:
paramParser.ignore('#' + pp.restOfLine)
with open(filename) as fin:
    data = paramParser.searchString(fin.read())[0]
    print(data.dump())

[['debug', False], ['shape', 2.3], ['length', 25361.15], ['path_1', 'C:\\\\This\\is\\a\\long\\path\\with some space in it\\data.txt'], ['description', 'raw values can have multiple lines, but additional lines must start\nwith a whitespace which is automatically skipped'], ['parent', None], ['names_', ['debug', 'shape', 'length', 'path_1', 'description', 'parent']], ['unit_', {'shape': 'mm^-1', 'length': 'mm'}]]
- debug: False
- description: 'raw values can have multiple lines, but additional lines must start\nwith a whitespace which is automatically skipped'
- length: 25361.15
- names_: ['debug', 'shape', 'length', 'path_1', 'description', 'parent']
- parent: None
- path_1: 'C:\\\\This\\is\\a\\long\\path\\with some space in it\\data.txt'
- shape: 2.3
- unit_: {'shape': 'mm^-1', 'length': 'mm'}
[0]:
  ['debug', False]
[1]:
  ['shape', 2.3]
[2]:
  ['length', 25361.15]
[3]:
  ['path_1', 'C:\\\\This\\is\\a\\long\\path\\with some space in it\\data.txt']
[4]:
  ['description', 'raw values ca

# Разбор таблиц

Элемент `Forward` позволяет определять правила разбора на лету. Например, для разбора таблиц данных нужно, чтобы **количество столбцов данных было такое же, как количество столбцов в заголовке**. Разберем таблицу, заданную по формату, где количество столбцов заранее не известно:

* Имена не могут содержать пробелы
* Units обязательны в каждом столбце
* Значение могут быть стандартными питоновскими значениями или строками в кавычках, а raw string не должны содержать пробелы или `[`.

### Элемент Forward

[Forward](https://pythonhosted.org/pyparsing/pyparsing.Forward-class.html) используется, когда определение элемента будет доопределено позже.

Оператор `<<` переопределяет элемент `Forward`.

In [14]:
# We define ends-of-line and what kind of values we expect in tables
EOL          = pp.LineEnd().suppress()
tabValueDef  = pyValue | pp.CharsNotIn('[ \t\r\n').setWhitespaceChars(" \t")

# We define how to detect the first line, which is a header line
# following lines will be defined later
firstLine    = pp.Group(pp.OneOrMore(keyNameWithoutSpace)+EOL)
unitLine     = pp.Forward()
tabValueLine = pp.Forward()

def defineColNumber(t):
    """ Define unitLine and tabValueLine to match the same number of columns than
    the header line"""
    nbcols = len(t.header)
    unitLine      << pp.Group( unitDef*nbcols + EOL)
    tabValueLine  << pp.Group( tabValueDef*nbcols + EOL)

tableColDef = (   firstLine('header').setParseAction(defineColNumber)
                + unitLine('unit')
                + pp.Group(pp.OneOrMore(tabValueLine))('data')
              )

In [15]:
text = '''STATION         PRECIPITATION   T_MAX_ABS  T_MIN_ABS
(it)                     (mm)    (C)        (C)       # Columns must have a unit
Ajaccio                 64.8    18.8E+0    -2.6
Auxerre                 49.6    16.9E+0    Nan       # Here is a Nan
Bastia                  114.2   20.8E+0    -0.9
'''

tableColDef.ignore('#' + pp.restOfLine)
result = tableColDef.parseString(text)
result

ParseResults([ParseResults(['station', 'precipitation', 't_max_abs', 't_min_abs'], {}), ParseResults(['it', 'mm', 'C', 'C'], {}), ParseResults([ParseResults(['Ajaccio', 64.8, 18.8, -2.6], {}), ParseResults(['Auxerre', 49.6, 16.9, nan], {}), ParseResults(['Bastia', 114.2, 20.8, -0.9], {})], {})], {'header': ['station', 'precipitation', 't_max_abs', 't_min_abs'], 'unit': ['it', 'mm', 'C', 'C'], 'data': [['Ajaccio', 64.8, 18.8, -2.6], ['Auxerre', 49.6, 16.9, nan], ['Bastia', 114.2, 20.8, -0.9]]})

In [27]:
def formatBloc(t):
    """ Format the result to have a list of (key, values) easily usable
    with Dict and transform data into array

    Add two fields :
        names_ : the list of column names found
        units_ : a dict in the form {key : unit}
    """
    columns = []

    # store names and units names
    names = t.header
    units   = {}

    transposedData = zip(*t.data)
    for header, unit, data in zip(t.header, t.unit, transposedData):
        units[header] = unit
        columns.append(pp.ParseResults([header, np.array(data)]))

    columns.append(pp.ParseResults(['names_', names]))
    columns.append(pp.ParseResults(['unit_'   , units  ]))

    return columns

tableColParser = pp.Dict(tableColDef.setParseAction(formatBloc))

In [28]:
tableColParser.ignore('#' + pp.restOfLine)
with open(filename) as fin:
    data = tableColParser.searchString(fin.read())[0]
print(data.dump())

[['station', array(['Ajaccio', 'Auxerre', 'Bastia'], dtype='<U7')], ['precipitation', array([ 64.8,  49.6, 114.2])], ['t_max_abs', array([18.8, 16.9, 20.8])], ['names_', ['station', 'precipitation', 't_max_abs', 't_min_abs']], ['unit_', {'station': 'mm', 'precipitation': 'C', 't_max_abs': 'C'}]]
- names_: ['station', 'precipitation', 't_max_abs', 't_min_abs']
- precipitation: array([ 64.8,  49.6, 114.2])
- station: array(['Ajaccio', 'Auxerre', 'Bastia'], dtype='<U7')
- t_max_abs: array([18.8, 16.9, 20.8])
- unit_: {'station': 'mm', 'precipitation': 'C', 't_max_abs': 'C'}
[0]:
  ['station', array(['Ajaccio', 'Auxerre', 'Bastia'], dtype='<U7')]
[1]:
  ['precipitation', array([ 64.8,  49.6, 114.2])]
[2]:
  ['t_max_abs', array([18.8, 16.9, 20.8])]
[3]:
  ['names_', ['station', 'precipitation', 't_max_abs', 't_min_abs']]
  [0]:
    names_
  [1]:
    ['station', 'precipitation', 't_max_abs', 't_min_abs']
[4]:
  ['unit_', {'station': 'mm', 'precipitation': 'C', 't_max_abs': 'C'}]


# Приложение. Цитаты из докуменатации

## pyparsing.Dict

https://pythonhosted.org/pyparsing/pyparsing.Dict-class.html

In [16]:
data_word = pp.Word(pp.alphas)
label = data_word + pp.FollowedBy(':')
attr_expr = pp.Group(label + pp.Suppress(':') + pp.OneOrMore(data_word).setParseAction(' '.join))

text = "shape: SQUARE posn: upper left color: light blue texture: burlap"
attr_expr = (label + pp.Suppress(':') + pp.OneOrMore(data_word, stopOn=label).setParseAction(' '.join))

# print attributes as plain groups
schema = pp.OneOrMore(attr_expr)
result = schema.parseString(text)
result
#print(result.dump())

ParseResults(['shape', 'SQUARE', 'posn', 'upper left', 'color', 'light blue', 'texture', 'burlap'], {})

In [18]:
# instead of OneOrMore(expr), parse using Dict(OneOrMore(Group(expr))) - Dict will auto-assign names
schema = pp.Dict(pp.OneOrMore(pp.Group(attr_expr)))
result = schema.parseString(text)
#print(result.dump())
result

ParseResults([ParseResults(['shape', 'SQUARE'], {}), ParseResults(['posn', 'upper left'], {}), ParseResults(['color', 'light blue'], {}), ParseResults(['texture', 'burlap'], {})], {'shape': 'SQUARE', 'posn': 'upper left', 'color': 'light blue', 'texture': 'burlap'})

In [ ]:
# access named fields as dict entries, or output as dict
print(result['shape'])        
print(result.asDict())